In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib

/Users/leeyn/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
def load_data(file):
    data = pd.read_csv(file, sep="\n",header=None)
    return data
good = load_data('normal.csv')
bad = load_data('xssed.csv')
print(len(good))
print(len(bad))


31407
33426


In [3]:
def get_label_vec(url):
    script = str(url).lower().count("script")
    java = str(url).lower().count("java")
    iframe = str(url).lower().count("iframe")
    quot_1 = str(url).lower().count("<")
    quot_2 = str(url).lower().count(">")
    quot_3 = str(url).lower().count("\"")
    quot_4 = str(url).lower().count("\'")
    quot_5 = str(url).lower().count("%")
    quot_6 = str(url).lower().count("(")
    quot_7 = str(url).lower().count(")")
    vec = [script,java,iframe,quot_1,quot_2,quot_3,quot_4,quot_5,quot_6,quot_7]
    return vec

get_label_vec("q=<script>document.write(String.fromCharCode(60, 105, 102, 114, 97,<br/>109, 101, 32, 115, 114, 99, 61, 34, 104, 116, 116, 112, 58, 47, 47, 120, 115, 115, 101, 100, 46, 99,<br/>111, 109, 34, 62))</script>")

[2, 0, 0, 4, 4, 0, 0, 0, 2, 2]

In [4]:
from sklearn.model_selection import train_test_split
def get_train():
    from sklearn import preprocessing
    x = pd.concat([good[0],bad[0]])
    y = [0]*len(good)+[1]*len(bad)
    vec = []
    for i in x:
        vec.append(get_label_vec(i))
    a=preprocessing.scale(vec)
    x_train, x_test, y_train, y_test=train_test_split(a,y,test_size=0.3)
    return x_train, x_test, y_train, y_test
        
    
  
    

In [5]:
from sklearn import svm
from sklearn.metrics import classification_report
def do_svm(x_train,x_test,y_train,y_test):
    clf = svm.SVC(kernel='rbf',decision_function_shape="ovo",random_state=0,gamma=0.001,C=100, probability=True)
    clf.fit(x_train, y_train)
    joblib.dump(clf,'svm.m')
    y_pred = clf.predict(x_test)
    print("svm训练结果")
    print(classification_report(y_test, y_pred))


x_train, x_test, y_train, y_test=get_train() 
do_svm(x_train,x_test,y_train,y_test)

svm训练结果
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      9450
           1       1.00      0.96      0.98     10000

    accuracy                           0.98     19450
   macro avg       0.98      0.98      0.98     19450
weighted avg       0.98      0.98      0.98     19450



In [7]:
def test(model,url):
    clf = joblib.load(model)
    vec = get_label_vec(url)
    result = clf.predict(np.array(vec).reshape(1,-1))

    if result == 0:
        print("Normal")
    else:
        print("XSS")
test("svm.m","search_text=/""><script>alert(1)</script>&I1.x=12&I1.y=14")

XSS
